# Test plane extraction for a single point cloud

In [87]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import time
import open3d as o3d

import planeslam.general as general
from planeslam.mesh import LidarMesh
from planeslam.scan import Scan
from planeslam.clustering import cluster_mesh_graph_search, plot_clusters
from planeslam.extraction import scan_from_clusters
import planeslam.io as io

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Read Point Cloud from pcd file

In [82]:
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)
P = PCs[0]

In [83]:
# Convert points to ENU
P = general.NED_to_ENU(P)

# Downsample the points
P = general.downsample(P, factor=2, axis=0)

In [85]:
# # Plot the points 
general.plot_pc(P)

Cluster the points

In [77]:
# Create the mesh
start_time = time.time()
mesh = LidarMesh(P)
print("elapsed time: ", time.time() - start_time)

elapsed time:  0.040997982025146484


In [78]:
start_time = time.time()
mesh.prune(edge_len_lim=10)
print("elapsed time: ", time.time() - start_time)

elapsed time:  0.014009237289428711


In [79]:
# Cluster the mesh with graph search
start_time = time.time()
clusters, avg_normals = cluster_mesh_graph_search(mesh)
print("elapsed time: ", time.time() - start_time)

elapsed time:  0.0429997444152832


In [80]:
#%lprun -f cluster_mesh_graph_search cluster_mesh_graph_search(mesh)

In [81]:
# # Plot mesh
mesh.plot()

In [88]:
# Plot clusters
plot_clusters(P, mesh, clusters)

Extract planes

In [63]:
planes, vertices, faces = scan_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes, vertices, faces)

In [92]:
# Plot the scan
scan.plot()